In [1]:
import os,sys,inspect
import json
import torch
currentdir = os.path.dirname(os.getcwd())
print(currentdir)
# parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(currentdir)
print("root Folder Absoloute path: ", root)


import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

import __main__

__main__.__file__="main.py"

/home/hfaghihi/Framework/fix/DomiKnowS/examples
root Folder Absoloute path:  /home/hfaghihi/Framework/fix/DomiKnowS


In [2]:
with open("data/tracker.log", "r") as file:
    tracker = []
    for line in file:
        tracker.append(json.loads(line))
print(len(tracker))
print(tracker[0].keys())
state2idx = {'O_C': 0, 'O_D': 1, 'E': 2, 'M': 3, 'C': 4, 'D': 5}

54
dict_keys(['para_id', 'location_cand', 'states', 'loc_spans', 'boundaries', 'states_mapping', 'states_in_tokens', 'spans', 'roberta_tokens', 'states_annotation', 'roberta_bounds', 'participants', 'sentence_paragraph', 'sentence_texts', 'trips_annotation', 'actions', 'ltype', 'location_entity_cands', 'verb_mentions', 'entity_mentions', 'computed_boundaries', 'tags', 'location_indexes', 'net_results'])


In [3]:
import torch

torch.tensor(tracker[10]['net_results']['tag_logits']).shape

torch.Size([2, 5, 6])

In [4]:
import json
import torch

with open("data/train_propara_roberta_version_trips.json", "r") as file:
    data = json.load(file)
print(len(data))
updated_data = []
lines = 0
for item in tracker:
    lines += len(item["sentence_texts"])
    try:
        temp = item.copy()
        new_state = []
        location_types = []
        for state_id, states in enumerate(temp['states']):
            temp2 = ["-", ]
            
            ltemp = []
            if states[0] == "?":
                ltemp.append("Unknown")
            elif states[0] == "-":
                ltemp.append("-")
            else:
                ltemp.append("Known")
                
            prev_loc = states[0]
            for loc in states[1:]:
                if loc == "?":
                    ltemp.append("Unknown")
                elif loc == "-":
                    ltemp.append("-")
                else:
                    ltemp.append("Known")

                if loc == prev_loc:
                    temp2.append("No change")
                elif loc != "-" and prev_loc == "-":
                    temp2.append("Create")
                elif loc == "-" and prev_loc != "-":
                    temp2.append("Destroy")
                else:
                    temp2.append("Move")
                prev_loc = loc
            new_state.append(temp2)
            location_types.append(ltemp)
        temp['actions'] = new_state
        temp['ltype'] = location_types
        

#         new_annotation = []
#         for state in temp['states_annotation']:
#             temp2 = state
#             for loc in temp2:
#                 if loc[0] == "?":
#                     loc[2] = item['boundaries'][-1][1] + 1
#                     loc[3] = item['boundaries'][-1][1] + 1
#             new_annotation.append(temp2)

#         temp['new_annotation'] = new_annotation
    except:
        print(item)
        raise
    updated_data.append(temp)
    
for item in updated_data:
    steps = len(item['sentence_paragraph'])
    entities = len(item['participants'])
    probs = torch.tensor(item['net_results']['tag_logits']).softmax(dim=-1)
    probs[:, :, 0] = probs[:, :, 0] + probs[:, :, 1] + probs[:, :, 2]
    probs = torch.cat((probs[:, :, 0:1], probs[:, :, 3:]), dim=-1)
    item['action_probs'] = probs.tolist()
#     tprobs = torch.softmax(torch.randn(steps, entities, 4), dim=-1)
#     item['Taction_probs'] = tprobs.tolist()
    
with open("data/train.json", "w") as file:
    json.dump(updated_data, file)

391


In [5]:
with open("data/train.json", "r") as file:
    data = json.load(file)
print(len(data))

54


In [6]:
from regr.data.reader import RegrReader
import torch


class ProparaReader(RegrReader):
    def getProcedureIDval(self, item):
        return [item["para_id"]]

    def getEntitiesval(self, item):
        return [' '.join(item['participants'])]
        
    def getEntityval(self, item):
        return item["participants"]
    
    def getContextval(self, item):
        return [item['sentence_paragraph']]

    def getStepval(self, item):
        sentences = item["sentence_texts"]
        return  sentences
    
    def getActionval(self, item):
        return torch.tensor(item['action_probs'])
    
    def getbeforeval(self, item):
        b1s = []
        b2s = []
#         for step in range(len(item["sentence_texts"])):
#             b1 = torch.zeros(len(item["sentence_texts"]))
#             b1[step] = 1
#             for step1 in range(len(item["sentence_texts"])):
#                 b2 = torch.zeros(len(item["sentence_texts"]))
#                 b2[step1] = 1
#                 b1s.append(b1)
#                 b2s.append(b2)
#         return torch.stack(b1s), torch.stack(b2s)
        for step in range(len(item['sentence_texts'])):
            for step1 in range(len(item['sentence_texts'])):
                if step < step1:
                    b1 = torch.zeros(len(item["sentence_texts"]))
                    b1[step] = 1
                    b2 = torch.zeros(len(item["sentence_texts"]))
                    b2[step1] = 1
                    b1s.append(b1)
                    b2s.append(b2)
                    
                    
        return torch.stack(b1s), torch.stack(b2s)
        

    def getbefore_trueval(self, item):
        num_steps = len(item["sentence_texts"])
        values = torch.zeros(num_steps * num_steps)
        for step in range(len(item["sentence_texts"])):
            for step1 in range(step + 1, len(item["sentence_texts"])):
                values[(step * num_steps) + step1] = 1
        return values
    
    def getexact_beforeval(self, item):
        b1s = []
        b2s = []
        for step in range(len(item['sentence_texts'])):
            step1 = step + 1
            if step1 < len(item['sentence_texts']):
                b1 = torch.zeros(len(item["sentence_texts"]))
                b1[step] = 1
                b2 = torch.zeros(len(item["sentence_texts"]))
                b2[step1] = 1
                b1s.append(b1)
                b2s.append(b2)
                    
                    
        return torch.stack(b1s), torch.stack(b2s)
    
    def getLocationsval(self, item):
        return [item['location_cand']]
    
    def getLocationval(self, item):
        return item['location_cand']
    
    def getLocationLabelval(self, item):
        return torch.tensor(item['net_results']['location'])
                
    
#     def getTActionval(self, item):
#         actions = []
#         for step, step_text in enumerate(item['sentence_texts']):
#             actions.append([])
#             for eid, entity in enumerate(item['participants']):
#                 actions[-1].append(item['Taction_probs'])

#         return torch.tensor(actions)


In [7]:
train_reader = ProparaReader("data/train.json")
runner = iter(train_reader)
sample = next(runner)
print(sample['before'][0].shape)
print(sample['before'][1].shape)
print(sample['before'])
print(sample['exact_before'][0].shape)
# print(sample['before_true'])
print(sample['Locations'])
print(sample['LocationLabel'].shape)
# select = torch.where(sample['before_true']==1, True, False)
# print(select)

# print(len(sample['Step']))
# print(sample['before'])
# print(select.shape)
# print(sample['before'][0].shape)
# print(torch.masked_select(sample['before'][0], select))
# assert sample['before'][0].shape == sample['before'][1].shape

torch.Size([15, 6])
torch.Size([15, 6])
(tensor([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.]]), tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
     

In [8]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL
from regr.graph import EnumConcept


Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    procedure = Concept(name="procedure")
    
    context = Concept(name="context")
    entities = Concept(name="entities")
    locations = Concept(name='locations')
    
    (procedure_context, procedure_entities, procedure_locations) = procedure.has_a(context, entities, locations)
    
    entity = Concept(name='entity')
    (entity_rel, ) = entities.contains(entity)
    
    step = Concept(name="step")
    (context_step, ) = context.contains(step)
    
    location = Concept(name="location")
    (loc_rel, ) = locations.contains(location)
    
    before = Concept(name="before")
    (before_arg1, before_arg2) = before.has_a(arg1=step, arg2=step)
    
    exact_before = Concept(name="exact_before")
    (ebefore_arg1, ebefore_arg2) = exact_before.has_a(earg1=step, earg2=step)
    
    action = Concept(name='action')
    (action_step, action_entity) = action.has_a(step=step, entity=entity)
    
    entity_location = Concept(name='entity_location')
    (lentity, lstep, llocation) = entity_location.has_a(lentity=entity, lstep=step, llocation=location)
    
    entity_location_label = entity_location(name='entity_location_label')
    
    
    action_label = action(name="action_label", ConceptClass=EnumConcept, values=["nochange", "move", "create", "destroy"])
    
    same_mention = Concept(name='same_mention')
    (same_entity, same_location) = same_mention.has_a(se1=entity, se2=location)
    
    
    
    
    ifL(
        # action a1 is destroy, i is a1's step and e is action entity
        andL(
            action_label.destroy('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
        orL(
            # step j associated with entity e, which is before step i cannot be associated with destroy action a2
            ifL(
                step('j', path=(('i', before_arg2.reversed, before_arg1))), 
                notL(action_label.destroy('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed))))
                ), 
            # or if  
            ifL(
                # step j1 which is before step i is associated with destroy action a2
                andL(
                    step('j1', path=('i', before_arg2.reversed, before_arg1)), 
                    action_label.destroy('a3', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
                    ), 
                # then exists step k associated with entity e, which is between step i and j1 associated with create action a3
                existsL(
                    andL(
                        step('k', path=(('j1', before_arg1.reversed, before_arg2), ('i', before_arg2.reversed, before_arg1))), 
                        action_label.create('a4', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
                        )
                    )
                )
            ), active = True
        ) 
    
    ifL(
        # action a1 is destroy, i is a1's step and e is action entity
        andL(
            action_label.destroy('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
#         orL(
            # step j associated with entity e, which is before step i cannot be associated with destroy action a2
        andL(
            step('j', path=(('i', before_arg2.reversed, before_arg1))), 
            notL(action_label.destroy('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed))))
            ), 
            # or if  
#             ifL(
#                 # step j1 which is before step i is associated with destroy action a2
#                 andL(
#                     step('j1', path=('i', before_arg1.reversed, before_arg2)), 
#                     action_label.destroy('a2', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
#                     ), 
#                 # then exists step k associated with entity e, which is between step i and j1 associated with create action a3
#                 existsL(
#                     andL(
#                         step('k', path=(('j1', before_arg2.reversed, before_arg1), ('i', before_arg1.reversed, before_arg2))), 
#                         action_label.create('a3', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
#                         )
#                     )
#                 )
#             ), 
            active = False
        )

    # ----------------------------
    
    ifL(
        # action a1 is create, i is a1's step and e is action entity
        andL(
            action_label.create('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
        orL(
            # step j associated with entity e, which is before step i cannot be associated with create action a2
            ifL(
                step('j', path=(('i', before_arg2.reversed, before_arg1))), 
                notL(action_label.create('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed))))
                ), 
            # or if  
            ifL(
                # step j1 which is before step i is associated with create action a2
                andL(
                    step('j1', path=('i', before_arg2.reversed, before_arg1)), 
                    action_label.create('a2', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
                    ), 
                # then exists step k associated with entity e, which is between step i and j1 associated with destroy action a3
                existsL(
                    andL(
                        step('k', path=(('j1', before_arg1.reversed, before_arg2), ('i', before_arg2.reversed, before_arg1))), 
                        action_label.destroy('a3', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
                        )
                    )
                )
            ), active = True
        )
    
    #  ------------
    
    ifL(
        # action a1 is move, i is a1's step and e is action entity
        andL(
            action_label.move('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity))
            ), 
        # then either
        orL(
            # step j associated with entity e, which is before step i cannot be associated with destroy action a2
            step('j', path=(('i', before_arg2.reversed, before_arg1))), 
            notL(action_label.destroy('a2', path=(('j', action_step.reversed), ('e', action_entity.reversed)))), 
            # or if  
            ifL(
                # step j1 which is before step i is associated with destroy action a2
                andL(
                    step('j1', path=('i', before_arg2.reversed, before_arg1)), 
                    action_label.destroy('a3', path=(('j1', action_step.reversed), ('e', action_entity.reversed)))
                    ), 
                # then exists step k associated with entity e, which is between step i and j1 associated with create action a3
                existsL(
                    andL(
                        step('k', path=(('j1', before_arg1.reversed, before_arg2), ('i', before_arg2.reversed, before_arg1))), 
                        action_label.create('a4', path=(('k', action_step.reversed), ('e', action_entity.reversed)))
                        )
                    )
                )
            ), active = True
        )
    
    
    ### If the action is move, then the location from step before should be different from the current step
    ifL(
        # action a1 is move, i is a1's step and e is action entity
        andL(
            action_label.move('a1'), 
            step('i', path=('a1', action_step)),
            entity('e', path=('a1', action_entity)),
            entity_location_label('x', path=(('i', lstep.reversed), ('e', lentity.reversed)))
            ), 
        andL(
            step('j', path=('i', ebefore_arg2.reversed, ebefore_arg1)),
            notL(entity_location_label('y', path=(('j', lstep.reversed), ('e', lentity.reversed), ('x', llocation, llocation.reversed))))
        ),
        active = True
    )
    
    ### There can only be one location for each entity at each step
    ifL(
        andL(
            entity('e'),
            step('i')
        ),
        atMostL( 
            entity_location_label('x', path=(('i', lstep.reversed), ('e', lentity.reversed))), 1
        ),
        active = True
    )
    
    
    # if 
#     ifL(
#         # action a1 is destroy, i is a1's step and e is action entity
#         andL(
#             action_label.destroy('a1'), 
#             step('i', path=('a1', action_step)),
#             entity('e', path=('a1', action_entity))
#             ), 
        
#         # step j associated with entity e, which is before step i cannot be associated with destroy action a2
#         andL(
#             step('j', path=(('e', action_entity.reversed, action_step), ('i', before_arg1.reversed, before_arg2))), 
#             notL(action_label.destroy('a2', path=('j', action_step.reversed)))
#             ), 
#         active = False
#         ) 




Log file for dataNode is in: /home/hfaghihi/Framework/fix/DomiKnowS/examples/ModularPropara/logs/datanode.log


main.py:29: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.
  
main.py:32: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('earg1', 'step'), ('earg2', 'step')]) is used.
  
main.py:35: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('step', 'step'), ('entity', 'entity')]) is used.
  step[procedure_contain_step, "text"] = JoinReaderSensor(procedure["id"], keyword="steps")
main.py:38: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('lentity', 'entity'), ('lstep', 'step'), ('llocation', 'location')]) is used.
  


In [9]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

class JointFunctionalReaderSensor(JointSensor, FunctionalReaderSensor):
    pass


procedure['id'] = ReaderSensor(keyword='ProcedureID')
context['text'] = ReaderSensor(keyword='Context')
entities['text'] = ReaderSensor(keyword='Entities')
locations['text'] = ReaderSensor(keyword='Locations')

def make_procedure(arg1m, arg2m, arg3m, data):
    total_procedures = len(arg1m) * len(arg2m) * len(arg3m)
    rel_links1 = torch.ones(total_procedures, len(arg1m))
    rel_links2 = torch.ones(total_procedures, len(arg2m))
    rel_links3 = torch.ones(total_procedures, len(arg3m))
#     past1 = 0
#     past2 = 0
#     for i in range(total_procedures):
#         rel_links1[i, past2: past2 + len(arg2m)] = 1
#         past2 = past2 + len(arg2m)
#         rel_links2[i, past1: past1 + len(arg1m)] = 1
#         past1 = past1 + len(arg1m)

#     for i in range(len(arg1m)):
#         rel_links1[0, i: (i+1)*(len(arg2m) * len(arg3m))] = 1
        
    

    return rel_links1, rel_links2, rel_links3


procedure[procedure_context.reversed, procedure_entities.reversed, procedure_locations.reversed] = JointFunctionalReaderSensor(context['text'], entities['text'], locations['text'], keyword="ProcedureID", forward=make_procedure)

def read_initials(*prev, data):
    number = len(data)
    rel_links = torch.ones(number, 1)
    indexes = [i for i in range(number)]
#     print(rel_links, data, indexes)
    return rel_links, data, indexes

entity[entity_rel, 'text', 'index'] = JointFunctionalReaderSensor(entities['text'], keyword='Entity', forward=read_initials)

step[context_step, 'text', 'index'] = JointFunctionalReaderSensor(context['text'], keyword='Step', forward=read_initials)

location[loc_rel, 'text', 'index'] = JointFunctionalReaderSensor(locations['text'], keyword='Location', forward=read_initials)



def make_before_connection(*prev, data):
    return data[0], data[1]

before[before_arg1.reversed, before_arg2.reversed] = JointFunctionalReaderSensor(step['text'], keyword='before', forward=make_before_connection)

exact_before[ebefore_arg1.reversed, ebefore_arg2.reversed] = JointFunctionalReaderSensor(step['text'], keyword='exact_before', forward=make_before_connection)


def make_actions(r1, r2, entities, steps):
#     print(r1, r2)
    all_actions = len(steps) * len(entities)
    link1 = torch.zeros(all_actions, len(steps))
    link2 = torch.zeros(all_actions, len(entities))
    for i in range(len(entities)):
        link2[i*len(steps):(i+1)*len(steps),i] = 1

    for j in range(all_actions):
        link1[j, j%len(steps)] = 1

#     print(link1, link2)
#     print(link1.shape, link2.shape)
#     print("steps: ", len(steps))
#     print("entities: ", len(entities))
    return link1, link2

action[action_step.reversed, action_entity.reversed] = JointSensor(entity[entity_rel], step[context_step], entity['index'], step['index'], forward=make_actions)

def make_same_mentions(r1, r2, entities, locations):
    matches = []
    for eid, ent in enumerate(entities):
        for lid, loc in emumerate(locations):
            if ent == loc:
                matches.append(eid, lid)
    link1 = torch.zeros(len(matches), len(entities))
    link2 = torch.zeros(len(matches), len(locations))
    
    for mid, match in enumerate(matches):
        link1[mid][match[0]] = 1
        link2[mid][match[1]] = 1
        
    return link1, link2
                

same_mention[same_entity.reversed, same_location.reversed] = JointSensor(entity[entity_rel], location[loc_rel], entity['text'], location['text'], forward=make_same_mentions)


def make_entity_locations(r1, r2, r3, entities, steps, locations):
#     print(r1, r2)
    all_actions = len(steps) * len(entities) * len(locations)
    link2 = torch.zeros(all_actions, len(steps))
    link1 = torch.zeros(all_actions, len(entities))
    link3 = torch.zeros(all_actions, len(locations))
    # for i in range(len(entities)):
    #     link2[i*len(steps):(i+1)*len(steps),i] = 1

    # for j in range(all_actions):
    #     link1[j, j%len(steps)] = 1

    for i in range(len(entities)):
        link1[i*len(steps)*len(locations):(i+1)*len(steps)*len(locations), i] = 1

    for i in range(len(entities)):
        for j in range(len(steps)):
            start = i*len(steps)*len(locations)
            link2[start+(len(locations)*j): start +((j+1)*len(locations)), j] = 1

    for i in range(len(entities)):
        for j in range(len(steps)):
            for k in range(len(locations)):
                start = i*len(steps)*len(locations) + (j*len(locations))
                link3[start+k] = 1

#     print(link1, link2)
#     print(link1.shape, link2.shape)
#     print("steps: ", len(steps))
#     print("entities: ", len(entities))
    return link1, link2, link3


entity_location[lentity.reversed, lstep.reversed, llocation.reversed] = JointSensor(entity[entity_rel], step[context_step], location[loc_rel], entity['index'], 
                                                                                    step['index'], location['index'], forward=make_entity_locations)

def read_location_labels(*prevs, data):
#     print(prevs[0].shape, prevs[1].shape)
#     print(data.shape)
    c = torch.softmax(data, dim=-1)
    d = c.repeat(1, 1, 2)
    d = d.view(c.shape[0], c.shape[1], c.shape[2], 2)
    for k in range(d.shape[2]):
        d[:, :, k, 1] = 1 - d[:, :, k, 0]
#     print(c.shape)

    d = d.view(c.shape[0] * c.shape[1] * c.shape[2], 2)
    return d

entity_location[entity_location_label] = FunctionalReaderSensor(lentity.reversed, lstep.reversed, llocation.reversed, keyword="LocationLabel", forward=read_location_labels)

def read_labels(*prevs, data):
#     print(prevs[0].shape, prevs[1].shape)
#     print(data.shape)
    c = data.view(data.shape[0] * data.shape[1], data.shape[2])
#     print(c.shape)
    return c
#     pass

action[action_label] = FunctionalReaderSensor(action_step.reversed, action_entity.reversed, keyword="Action", forward=read_labels)


In [10]:
from regr.program import POIProgram, IMLProgram, SolverPOIProgram
from regr.program.primaldualprogram import PrimalDualProgram
from regr.program.model.pytorch import SolverModel
from regr.program.metric import MacroAverageTracker, PRF1Tracker, PRF1Tracker, DatanodeCMMetric
from regr.program.loss import NBCrossEntropyLoss, NBCrossEntropyIMLoss

In [11]:
program = SolverPOIProgram(graph, poi=(procedure, before, action, action_label, exact_before, entity_location, entity_location_label), 
                               inferTypes=['ILP', 'local/argmax'], 
                               loss=MacroAverageTracker(NBCrossEntropyLoss()), 
                               metric={'ILP':PRF1Tracker(DatanodeCMMetric()),'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})

In [12]:
lbp = program

dataset = ProparaReader(file="data/train.json")  # Adding the info on the reader


#     lbp.test(dataset, device='auto')
all_updates = []
for datanode in lbp.populate(dataset, device="cpu"):
#     tdatanode = datanode.findDatanodes(select = context)[0]
#     print(len(datanode.findDatanodes(select = context)))
#     print(tdatanode.getChildDataNodes(conceptName=step))
    datanode.inferILPResults(action, fun=None)
    final_output = {
        "id": datanode.getAttribute("id"),
        "steps": [],
        "actions": [],
        "steps_before": [],
        "actions_before": [],
    }

    entities_instances = datanode.findDatanodes(select=entity)
#     print(len(entities))
    steps_instances = datanode.findDatanodes(select=step)
    actions = datanode.findDatanodes(select=action)
#         print('a')

    for step_instance in steps_instances:
        a = step_instance.getAttribute('index')
        # rel = step.getRelationLinks(relationName=before)
        # print(rel)
    # print(len(steps_instances), "\n")
    for action_info in datanode.findDatanodes(select=action):
        c = action_info.getAttribute(action_label, "ILP")
        final_output["actions"].append(c)
        c1 = action_info.getAttribute(action_label)
        final_output["actions_before"].append(c1)

    final_output['actions'] = torch.stack(final_output['actions'])
    final_output['actions'] = final_output['actions'].view(len(entities_instances), len(steps_instances), 4)

    final_output['actions_before'] = torch.stack(final_output['actions_before'])
    final_output['actions_before'] = final_output['actions_before'].view(len(entities_instances), len(steps_instances), 4)

    all_updates.append(final_output)

Log file for ilpOntSolver is in: /home/hfaghihi/Framework/fix/DomiKnowS/examples/ModularPropara/logs/ilpOntSolver.log
Log file for ilpOntSolverTime is in: /home/hfaghihi/Framework/fix/DomiKnowS/examples/ModularPropara/logs/ilpOntSolver.log
Academic license - for non-commercial use only - expires 2022-09-15


INFO:gurobipy.gurobipy:Academic license - for non-commercial use only - expires 2022-09-15


Using license file /home/hfaghihi/gurobi.lic


INFO:gurobipy.gurobipy:Using license file /home/hfaghihi/gurobi.lic


IndexError: list index out of range

In [13]:
count = 0
for item in all_updates:
    if not torch.equal(item['actions_before'].argmax(dim=-1), item['actions'].argmax(dim=-1)):
        print(item['actions_before'].argmax(dim=-1))
        print(item['actions'].argmax(dim=-1))
        print(count)
        print("----" * 5, '\n')
    count += 1
# all_updates[number]['actions_before'].argmax(dim=-1)

tensor([[3, 3, 3, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0],
        [0, 0, 2, 0, 3, 0, 0],
        [0, 0, 2, 0, 3, 0, 0],
        [0, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 2]])
tensor([[3, 2, 3, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0],
        [0, 0, 2, 0, 3, 0, 0],
        [0, 0, 2, 0, 3, 0, 0],
        [0, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 2]])
4
-------------------- 

tensor([[0, 0, 0, 3, 3, 0, 0],
        [0, 2, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 3, 0, 0],
        [0, 0, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 0, 2]])
tensor([[0, 0, 0, 3, 0, 0, 0],
        [0, 2, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 3, 0, 0],
        [0, 0, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 0, 2]])
6
-------------------- 

tensor([[0, 3, 3, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0,

In [ ]:
number = 4
all_updates[number]['actions_before'].argmax(dim=-1)

In [ ]:
all_updates[number]['actions'].argmax(dim=-1)

In [14]:
idx2state = {0: 'NoChange', 1: 'Move', 2: 'Create', 3: 'Destroy'}

In [15]:
def evaluate(
        probability, data_item
    ):
        all_predictions = {"actions": [], "locations": []}

        specific_correct = {
            "tags": 0,
        }
        specific_total = {
            "tags": 0,
        }

        keys = ["tags", 'location']

        for key in keys:
            if key == "tags":
                preds = torch.tensor(probability[key])

                idx2state = {0: 'O_C', 1: 'M', 2: 'C', 3: 'D'}
                original_idx2state = {0: 'O_C', 1: 'O_D', 2: 'E', 3: 'M', 4: 'C', 5: 'D'}

                for eid in range(preds.shape[0]):
                    all_predictions['actions'].append([])
                    for sid in range(preds.shape[1]):
                        all_predictions['actions'][-1].append(idx2state[preds[eid][sid].item()])

            elif key == "location":
                preds = probability[key]

                # if self.config.unknown_candidate:
                #     if len(grounding["candidates"]):
                #         preds = probability[key].squeeze(-1)
                #         preds = torch.softmax(preds, dim=-1)
                #         preds_score, preds_index = torch.max(preds, dim=-1)
                # else:
                #     if len(grounding["candidates"]):
                #         preds = probability[key].squeeze(-1)
                #         preds = torch.sigmoid(preds)
                #         preds_score, preds_index = torch.max(preds, dim=-1)
                #         for pid, pvalue in enumerate(preds_score):
                #             if pvalue <= 0.5:
                #                 preds_index[pid] = -1
                #     else:
                #         preds_index = []
                #         for pvalue in target:
                #             preds_index.append(-1)
                #         preds_index = torch.tensor(preds_index, device=self.device)


                preds = torch.softmax(torch.tensor(preds), dim=-1)
                preds_score, preds_index = torch.max(preds, dim=-1)
                preds_eval = preds_index.view(preds_index.shape[0]*preds_index.shape[1])

                for eid in range(preds.shape[0]):
                    all_predictions['locations'].append([])
                    for sid in range(preds.shape[1]):
                        if preds_index[eid][sid] == len(data_item['location_cand']):
                            all_predictions['locations'][-1].append("?")
                        else:
                            all_predictions['locations'][-1].append(data_item["location_cand"][preds_index[eid][sid]])

        return all_predictions

In [16]:
for item in data:
    up = [it for it in all_updates if it['id'] == item['para_id']][0]
    assert len(item['net_results']['tags']) == len(up['actions'].argmax(dim=-1).tolist())
    assert len(item['net_results']['tags'][0]) == len(up['actions'].argmax(dim=-1).tolist()[0])
    item['net_results']['tags'] = up['actions'].argmax(dim=-1).tolist()
#     print(item['net_results']['tags'], "\n")

In [17]:
csv_table = []
prediction_table = []
for item in data:
#     print(item['para_id'])
    all_predictions = evaluate(item['net_results'], item)
    step_predictions = []
    try:
        for sid in range(len(item['sentence_texts']) + 1):
            for evalid, entity_text in enumerate(item["participants"]):
                if sid != 0:
                    step_predictions.append(
                        {
                            "para_id": item["para_id"],
                            "step": sid,
                            "entity": entity_text,
                            "action": all_predictions["actions"][evalid][sid-1],
                            "location": all_predictions["locations"][evalid][sid],
                        }
                    )
                else:
                    step_predictions.append(
                        {
                            "para_id": item["para_id"],
                            "step": sid,
                            "entity": entity_text,
                            "location": all_predictions["locations"][evalid][sid],
                        }
                    )

        prediction_table.extend(step_predictions)
    except:
        print(sid, len(all_predictions['actions'][0]), evalid, len(item['sentence_texts']))
        raise
        

In [18]:
import csv
csv_file = (
    "./data/random.tsv"
)
with open(csv_file, "w", newline="") as f_output:
    tsv_output = csv.writer(f_output, delimiter="\t")
    for sample_pred in prediction_table:
        if "action" in sample_pred:
            dd = [
                str(sample_pred["para_id"]),
                sample_pred["step"],
                sample_pred["entity"],
                sample_pred["action"],
                "?",
                sample_pred["location"],
            ]
        else:
            dd = [
                str(sample_pred["para_id"]),
                sample_pred["step"],
                sample_pred["entity"],
                "-",
                "-",
                sample_pred["location"],
            ]
        tsv_output.writerow(dd)

In [19]:
from tools import get_koala_fix

In [20]:
def fix_tsv():
    preds = []
    with open(
        "./data/random.tsv"
    ) as csv_file:
        # with open('test.tsv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        for row in csv_reader:
            preds.append(row)

    fixed_pred = get_koala_fix(preds)

    csv_file_logical = "./data/logical.tsv"
    with open(csv_file_logical, "w", newline="") as f_output:
        tsv_output = csv.writer(f_output, delimiter="\t")
        for item in fixed_pred:
            item = [
                item["para_id"],
                item["step"],
                item["entity"],
                item["action"],
                item["before"],
                item["after"],
            ]
            tsv_output.writerow(item)

In [21]:
import csv
preds = []
with open(
    "./data/random.tsv"
) as csv_file:
    # with open('test.tsv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter="\t")
    for row in csv_reader:
        preds.append(row)
        
print(preds[0])

['37', '0', 'plant; animal', '-', '-', '?']


In [22]:
fix_tsv()